In [41]:
# coding: utf-8

# # Importing Module
# - pandas for formatting or provides table
# - seabourn is wrapper over matplotlib

# In[1]:

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import csv
from collections import defaultdict

import re
from datetime import datetime
#get_ipython().magic('matplotlib inline')

In [42]:
# In[4]:
gender = r'(\b[Mm]ale)|(\b[Ff]emale)|(\bFEMALE)|(\bMALE)'
#gender = 'Female'or'Male'or'female'or'male'or'FEMALE'or'MALE'

Date = r'(([A-Z0-9][A-Z0-9]?[/])?[A-Z0-9][A-Z0-9]?[/][A-Z0-9][A-Z0-9][A-Z0-9]?[A-Z0-9]?)|([A-Za-z][A-Za-z][A-Za-z]\s..?[,]\s....)'

DOB  = r'DOB|[Dd][aA][tT][eE]\s[oO][fF]\s[Bb][iI][rR][tT][hH]'

year=r'\b(19|20)\d{2}\b'

product_type = r'(\b[Pp]roduct\s[Tt]ype):\s?.*'

faceamount = r'(\b[Ff]ace\s?[Aa]mount:?\s?.*)'

term = r'[Tt][eE][rR][mM]'
term_amount=r'[0-9]*'

weight = r'((?:[kK][gG]\.?)[ ]*\d+(?:[.,]\d+)?|\d+(?:[.,]\d+)?[ ]*(?:[kK][gG]\.?))'


In [43]:
df = pd.read_csv("before_after.csv",encoding= "utf-8")
df["Before"][0]

'face updated\nhad bypass in 2010. \ntakes following medications:\n1. metoprolol (blood pressure) - diagnosed 2014 - takes 25 mg daily \n2. levothyroxine (thyroid) - diagnosed 2016 - takes 25 mg daily \n'

In [44]:
from nltk.corpus import stopwords,wordnet as wn
from nltk.tokenize import wordpunct_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re


# In[18]:

In [48]:
def rem_punt(doc):
    ans = re.sub('"|\\n|\(|\)|,|\.|[$!--+@#:]',' ',doc)
    ans = re.sub(' +',' ',ans)
    ans = ans.lower()
    return ans

stop_word = set(stopwords.words('english'))

def tokenize(document): 
    lemmy = []
    for sent in sent_tokenize(document):
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            #print(token,tag)
            if token in stop_word:
                 continue
            lemma = lemmatize(token, tag)
            lemmy.append(lemma)
    return lemmy

def lemmatize(token, tag):
    tag = {
          'N': wn.NOUN,
          'V': wn.VERB,
          'R': wn.ADV,
          'J': wn.ADJ
    }.get(tag[0], wn.NOUN)
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token, tag)

def regex(document):
    
    values = []
    #values.append('Product type')
    v = re.search(product_type,document, re.I | re.U)
    if v:
        values.append(v.group(0))
    else:
        values.append('-1')
        
    v = re.search(faceamount,document, re.I | re.U)
    w = re.search(term_amount, document, re.I | re.U)
    #values.append('Face Amount')
    if v:
        values.append(v.group(0))
    elif(re.search(w and term, document, re.I | re.U)):
        values.append(w.group(0))
    else:
        values.append('-1')
        
    v = re.search(gender,document, re.I | re.U)
    #values.append('Gender')
    if v:
        values.append(v.group(0))
    else:   
        values.append('-1')
        
    v = re.search(year,document, re.I | re.U)
    u = re.search(DOB,document, re.I | re.U)
    currentYear = datetime.now().year
    #values.append('Age')
    if v and u:
        values.append(currentYear-(int)(v.group(0)))
    else:   
        values.append('-1')
        
    v = re.search(weight,document, re.I | re.U)
    #values.append('Gender')
    if v:
        values.append(v.group(0))
    else:   
        values.append('-1')
         
        
    return values

In [46]:

df['RmNoise'] = df['Before'].apply(rem_punt)

df['Lemmitize'] = df['RmNoise'].apply(tokenize)

df['Regex'] = df['Before'].apply(regex)

df.to_csv('Script2data.csv',index=False, encoding = "utf-8")

In [49]:
# In[25]:

df = pd.read_csv('Script2data.csv')


# # Statistical Modeling 

# In[26]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer,LabelEncoder
from sklearn.metrics import accuracy_score,classification_report


# In[28]:

X = df['Lemmitize']
y = df['Offer']
#lab_y = LabelEncoder()
#y = lab_y.fit_transform(y)

In [50]:
y

0    A
1    B
2    A
3    B
Name: Offer, dtype: object

In [74]:
# In[29]:

X_train,X_test,y_train,y_test = train_test_split(X,y)


# In[30]:

vect = TfidfVectorizer(max_df=1.0, max_features=15000, min_df=0.001, use_idf=True , ngram_range=(1,3))

In [86]:
#from xgboost.sklearn import XGBClassifier
#model = XGBClassifier(nthread=4,n_estimators=1000)


# # Navie Bayes

# In[23]:

#from sklearn.naive_bayes import GaussianNB,MultinomialNB
#model = GaussianNB()


# # ExtraTree Classifier

# In[24]:

#from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
#model = RandomForestClassifier(n_estimators=600,n_jobs=-1)


# # SVM Classifier
# 

# In[25]:

#from sklearn.svm import SVC
#model = SVC()


# # Logistic Regression 

# In[26]:

#from sklearn.linear_model import LinearRegression,SGDClassifier,LogisticRegression
#model = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,SGDClassifier,LogisticRegression


In [88]:
model = XGBClassifier(nthread=4,n_estimators=1000)
clf = make_pipeline(vect,model)

clf.fit(X_train,y_train)
pd = clf.predict(X_test)
accuracy = accuracy_score(y_test, pd)
print(accuracy)
print ("=====Accuracy Score ", "{0:.2f}".format(metrics.accuracy_score(y_test, pd)*100), "%")

/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:


1.0
=====Accuracy Score  100.00 %


/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [89]:
model = GaussianNB()
clf = make_pipeline(vect,model)

clf.fit(X_train,y_train)
pd = clf.predict(X_test)
accuracy = accuracy_score(y_test, pd)
print(accuracy)
print ("=====Accuracy Score ", "{0:.2f}".format(metrics.accuracy_score(y_test, pd)*100), "%")

/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:


TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [90]:
model = RandomForestClassifier(n_estimators=600,n_jobs=-1)
clf = make_pipeline(vect,model)

clf.fit(X_train,y_train)
pd = clf.predict(X_test)
accuracy = accuracy_score(y_test, pd)
print(accuracy)
print ("=====Accuracy Score ", "{0:.2f}".format(metrics.accuracy_score(y_test, pd)*100), "%")

/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:


1.0
=====Accuracy Score  100.00 %


In [91]:
model = SVC()
clf = make_pipeline(vect,model)

clf.fit(X_train,y_train)
pd = clf.predict(X_test)
accuracy = accuracy_score(y_test, pd)
print(accuracy)
print ("=====Accuracy Score ", "{0:.2f}".format(metrics.accuracy_score(y_test, pd)*100), "%")

0.0
=====Accuracy Score  0.00 %


/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:119: UserWarning: indptr array has non-integer dtype (float64)
  % self.indptr.dtype.name)


In [92]:
model = LogisticRegression()
clf = make_pipeline(vect,model)

clf.fit(X_train,y_train)
pd = clf.predict(X_test)
accuracy = accuracy_score(y_test, pd)
print(accuracy)
print ("=====Accuracy Score ", "{0:.2f}".format(metrics.accuracy_score(y_test, pd)*100), "%")

1.0
=====Accuracy Score  100.00 %


/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:


In [58]:
# # Model Fitting

#t0 = time()

# In[33]:

#clf = make_pipeline(vect,model)


# In[34]:

#clf.fit(X_train,y_train)

#print ("training time:", round(time()-t0, 3), "s")
#t1 = time()

# In[40]:

#pd = clf.predict(X_test)
#print ("prediction time:", round(time()-t1, 3), "s")

#accuracy = accuracy_score(y_test, pd)
#print(accuracy)

/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:


1.0


In [93]:
pd

array(['A'], dtype=object)

In [94]:
X_test


0    ['face', 'update', 'bypass', '2010', 'take', '...
Name: Lemmitize, dtype: object